In [1]:
import pandas as pd
import boto3
import json
import configparser
import base64

## Read all the configurations parameters via 3 configuration files with extension cfg
### Credentials configurations
The credentials are saved in credentials.cfg file, including:
- IAM user credentials are in AWS section, it is the general credentials to create all aws services using python code via boto3 library:
```
[AWS]
KEY=
SECRET=
```
With KEY is the IAM user's access KEY and SECRET is the IAM user's secret key, each keys must be encoded into base64 to ensure the best security.
- The redshift cluster credentials are in REDSHIFT section, it is the redshift credentials to connect to redshift cluster:
```
[REDSHIFT]
USER=
PASSWORD=
```
With USER is the username of the redshift cluster, PASSWORD is the password of the redshift cluster and its value must be encoded to base64 to ensure the best security.
### Redshift configurations
The redshift configurations are saved in redshift.cfg file to create redshift cluster using python code via boto3 library, including:
```
[REDSHIFT]
CLUSTER_TYPE=
NUM_NODES=
NODE_TYPE=
CLUSTER_IDENTIFIER=
DB_NAME=
PORT=
IAM_ROLE_NAME=
REGION=
```
<strong>Based on the vital policies and requirements of aws, you must adjust these value mentioned above accordingly.</strong>
Descriptions:
CLUSTER_TYPE is the type of redshift cluster, it can be single-node or multi-node
NUM_NODES is the number of nodes in redshift cluster.
NODE_TYPE is the node type of redshift cluster, it can be only one of these values: ds2.xlarge, ds2.8xlarge, dc1.large, dc1.8xlarge, dc2.large, dc2.8xlarge, ra3.xlplus, ra3.4xlarge, ra3.16xlarge
CLUSTER_IDENTIFIER is the name of redshift-cluster.
DB_NAME is the database name from redshift cluster.
PORT is the redshift cluster port.
IAM_ROLE_NAME is the iam role name for redshift cluster.
REGION is the redshift cluster region, it can be any region of aws.

In [2]:
config = configparser.ConfigParser()

config.read(["credentials.cfg", "redshift.cfg"])

['credentials.cfg', 'redshift.cfg']

#### Load all configuration parameters.
After reading the credentials.cfg and redshift.cfg files successfully, load these configurations into pandas dataframe to verify that these configurations are loaded successfully.

In [3]:
KEY =  config.get("AWS", "KEY") # Iam user access key.
SECRET = config.get("AWS", "SECRET") # IAM user secret key

REGION = config.get("REDSHIFT", "REGION")
CLUSTER_TYPE = config.get("REDSHIFT", "CLUSTER_TYPE")
NUM_NODES = config.get("REDSHIFT", "NUM_NODES")
NODE_TYPE = config.get("REDSHIFT", "NODE_TYPE")
CLUSTER_IDENTIFIER = config.get("REDSHIFT", "CLUSTER_IDENTIFIER")
DB_NAME = config.get("REDSHIFT", "DB_NAME")
USER = config.get("REDSHIFT", "USER")
PASSWORD = config.get("REDSHIFT", "PASSWORD")
PORT = config.get("REDSHIFT", "PORT")
IAM_ROLE_NAME = config.get("REDSHIFT", "IAM_ROLE_NAME")

pd.DataFrame({
    "Configurations": [
        "KEY", "SECRET", "REGION", "CLUSTER_TYPE", "NUM_NODES", "NODE_TYPE", "CLUSTER_IDENTIFIER",
        "DB_NAME", "USER", "PASSWORD", "PORT", "IAM_ROLE_NAME"
    ],
    "Values": [
        KEY, SECRET, REGION,
        CLUSTER_TYPE, NUM_NODES, NODE_TYPE, CLUSTER_IDENTIFIER, DB_NAME,
        USER, PASSWORD, PORT, IAM_ROLE_NAME
    ]
})

,Configurations,Values
0,KEY,QUtJQVpDQVdOT1JHVFpUVE1ORVY=
1,SECRET,YlFvcXFwUGFCWjFZRVZRODlrRW9Sa2ZBUCtiVEx3STJheV...
2,REGION,us-west-2
3,CLUSTER_TYPE,multi-node
4,NUM_NODES,4
5,NODE_TYPE,dc2.large
6,CLUSTER_IDENTIFIER,sparkify-cluster
7,DB_NAME,sparkify
8,USER,quannct
9,PASSWORD,Mzk3OTk5X1F1YW4=


#### Utilizing the parameter configuration.
1. Decode the encoded access key and the encoded secret key.
2. Create multiple aws resources using access key, secret key and redshift region after loaded the configurations from credentials.cfg.

In [4]:
ec2 = boto3.resource("ec2", region_name=REGION,
                            aws_access_key_id=base64.b64decode(KEY).decode('utf-8'),
                            aws_secret_access_key=base64.b64decode(SECRET).decode('utf-8'))
s3 = boto3.resource("s3", region_name=REGION,
                          aws_access_key_id=base64.b64decode(KEY).decode('utf-8'),
                          aws_secret_access_key=base64.b64decode(SECRET).decode('utf-8'))
iam = boto3.client('iam', region_name=REGION,
                          aws_access_key_id=base64.b64decode(KEY).decode('utf-8'),
                          aws_secret_access_key=base64.b64decode(SECRET).decode('utf-8'))

redshift = boto3.client('redshift', region_name=REGION,
                                    aws_access_key_id=base64.b64decode(KEY).decode('utf-8'),
                                    aws_secret_access_key=base64.b64decode(SECRET).decode('utf-8'))

## Load multiple file from udacity-dend buckets defined from boto3 client above, here are the bucket files.
1. Song data: s3://udacity-dend/song_data
2. Log data: s3://udacity-dend/log_data
3. This third file s3://udacity-dend/log_json_path.json contains the meta information that is required by AWS to correctly load s3://udacity-dend/log_data

In [8]:
bucket=s3.Bucket('udacity-dend')
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/A/B/')] # Load song data files

print ("Song data files: ")
song_data_files

s3.Bucket.objectsCollectionManager(s3.Bucket(name='udacity-dend'), s3.ObjectSummary)
Song data files: 


['song-data/A/A/B/TRAABCL128F4286650.json',
 'song-data/A/A/B/TRAABDL12903CAABBA.json',
 'song-data/A/A/B/TRAABEV12903CC53A4.json',
 'song-data/A/A/B/TRAABFH128F92C812E.json',
 'song-data/A/A/B/TRAABGU12903CC8DCF.json',
 'song-data/A/A/B/TRAABHB12903CAFC2F.json',
 'song-data/A/A/B/TRAABHO12903D08576.json',
 'song-data/A/A/B/TRAABHP128F427AEC8.json',
 'song-data/A/A/B/TRAABIG128F9356C56.json',
 'song-data/A/A/B/TRAABIX128F92D6F94.json',
 'song-data/A/A/B/TRAABJS128F9325C99.json',
 'song-data/A/A/B/TRAABKD128F9302CE9.json',
 'song-data/A/A/B/TRAABLR128F423B7E3.json',
 'song-data/A/A/B/TRAABNV128F425CEE1.json',
 'song-data/A/A/B/TRAABOA128F933684A.json',
 'song-data/A/A/B/TRAABOG128F42955B1.json',
 'song-data/A/A/B/TRAABPG128F14774DD.json',
 'song-data/A/A/B/TRAABPO128F9312A56.json',
 'song-data/A/A/B/TRAABPQ128F428F0B0.json',
 'song-data/A/A/B/TRAABRX12903CC4816.json',
 'song-data/A/A/B/TRAABTB128F146BF24.json',
 'song-data/A/A/B/TRAABUQ128F92ECD79.json',
 'song-data/A/A/B/TRAABVM128F149

In [6]:
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log_data')] # Load log data files

log_data_files

['log_data/',
 'log_data/2018/11/2018-11-01-events.json',
 'log_data/2018/11/2018-11-02-events.json',
 'log_data/2018/11/2018-11-03-events.json',
 'log_data/2018/11/2018-11-04-events.json',
 'log_data/2018/11/2018-11-05-events.json',
 'log_data/2018/11/2018-11-06-events.json',
 'log_data/2018/11/2018-11-07-events.json',
 'log_data/2018/11/2018-11-08-events.json',
 'log_data/2018/11/2018-11-09-events.json',
 'log_data/2018/11/2018-11-10-events.json',
 'log_data/2018/11/2018-11-11-events.json',
 'log_data/2018/11/2018-11-12-events.json',
 'log_data/2018/11/2018-11-13-events.json',
 'log_data/2018/11/2018-11-14-events.json',
 'log_data/2018/11/2018-11-15-events.json',
 'log_data/2018/11/2018-11-16-events.json',
 'log_data/2018/11/2018-11-17-events.json',
 'log_data/2018/11/2018-11-18-events.json',
 'log_data/2018/11/2018-11-19-events.json',
 'log_data/2018/11/2018-11-20-events.json',
 'log_data/2018/11/2018-11-21-events.json',
 'log_data/2018/11/2018-11-22-events.json',
 'log_data/2018/11

In [7]:
meta_data_files = [filename.key for filename in bucket.objects.filter(Prefix='log_json_path.json')]

meta_data_files

['log_json_path.json']

## Create am IAM Role for Redshift Cluster
Create an IAM Role that makes Redshift able to access S3 bucket (ReadOnly)

In [11]:
try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )
    print(dwhRole)
except Exception as e:
    print(e)

{'Role': {'Path': '/', 'RoleName': 'redshift-iam-role', 'RoleId': 'AROAZCAWNORGYITM3DW3V', 'Arn': 'arn:aws:iam::622817277005:role/redshift-iam-role', 'CreateDate': datetime.datetime(2023, 2, 27, 16, 58, 59, tzinfo=tzutc()), 'AssumeRolePolicyDocument': {'Statement': [{'Action': 'sts:AssumeRole', 'Effect': 'Allow', 'Principal': {'Service': 'redshift.amazonaws.com'}}], 'Version': '2012-10-17'}}, 'ResponseMetadata': {'RequestId': 'deccbc6f-99a1-41fb-913b-1d0be11ab7be', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'deccbc6f-99a1-41fb-913b-1d0be11ab7be', 'content-type': 'text/xml', 'content-length': '790', 'date': 'Mon, 27 Feb 2023 16:58:59 GMT'}, 'RetryAttempts': 0}}


## Attach policy to new IAM role created above.
Attach all necessary policy into IAM ROLE.

In [12]:
print("1.2 Attaching Policy")

iam.attach_role_policy(RoleName=IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

## Verify that IAM Role is created successfully.
- Get ARN of IAM Role.

In [13]:
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::622817277005:role/redshift-iam-role


- Create a RedShift Cluster
- For complete arguments to `create_cluster`, see [docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html#Redshift.Client.create_cluster)

In [14]:
try:
    redshift_create_cluster_response = redshift.create_cluster(
    #HW
        ClusterType=CLUSTER_TYPE,
        NodeType=NODE_TYPE,
        NumberOfNodes=int(NUM_NODES),

        #Identifiers & Credentials
        DBName=DB_NAME,
        ClusterIdentifier=CLUSTER_IDENTIFIER,
        MasterUsername=USER,
        MasterUserPassword=base64.b64decode(PASSWORD).decode('utf-8'),
        
        #Roles (for s3 access)
        IamRoles=[roleArn]
    )
except Exception as e:
    print(e)

In [16]:
def get_redshift_settings(props):
    pd.set_option('display.expand_frame_repr', True)
    keys = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keys]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=CLUSTER_IDENTIFIER)['Clusters'][0]
get_redshift_settings(myClusterProps)

,Key,Value
0,ClusterIdentifier,sparkify-cluster
1,NodeType,dc2.large
2,ClusterStatus,available
3,MasterUsername,quannct
4,DBName,sparkify
5,Endpoint,{'Address': 'sparkify-cluster.cvidguchcuu7.us-...
6,VpcId,vpc-0b20b33ba0fad4013
7,NumberOfNodes,4


## Open an incoming  TCP port to access the cluster ednpoint
Do not run authorize the inbound rule of security group again if the default security group has been configured already.

In [18]:
vpc = ec2.Vpc(id=myClusterProps['VpcId'])
defaultSg = list(vpc.security_groups.all())[0]
print(defaultSg)
try:
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(PORT),
        ToPort=int(PORT)
    )
except Exception as e:
    print(e)

ec2.SecurityGroup(id='sg-00dfd05f7ce06ac62')


## *Describe* the cluster to see its status
- run this block several times until the cluster status becomes `Available`

<h2> 2.2 Take note of the cluster <font color='red'> endpoint and role ARN </font> </h2>
<font color='red'>DO NOT RUN THIS unless the cluster status becomes "Available". </font>

In [19]:
REDSHIFT_ENDPOINT = myClusterProps['Endpoint']['Address']
IAM_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("REDSHIFT_ENDPOINT: ", REDSHIFT_ENDPOINT)
print("IAM_ROLE_ARN: ", IAM_ROLE_ARN)

REDSHIFT_ENDPOINT:  sparkify-cluster.cvidguchcuu7.us-west-2.redshift.amazonaws.com
IAM_ROLE_ARN:  arn:aws:iam::622817277005:role/redshift-iam-role


# Clean up resources
<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE OR YOU DON'T USE THE AWS MANAGEMENT CONSOLE TO DELETE <br/></b>

## <strong><font color='red'>Remove IAM Role.</font></strong>

In [10]:
iam.detach_role_policy(RoleName=IAM_ROLE_NAME, PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess")
iam.delete_role(RoleName=IAM_ROLE_NAME)

{'ResponseMetadata': {'RequestId': '8b4dc15b-c855-405f-abf6-a11aa071b59a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8b4dc15b-c855-405f-abf6-a11aa071b59a',
   'content-type': 'text/xml',
   'content-length': '200',
   'date': 'Mon, 27 Feb 2023 16:58:49 GMT'},
  'RetryAttempts': 0}}

## <strong><font color='red'>Remove redshift cluster</font></strong>

In [21]:
redshift.delete_cluster( ClusterIdentifier=CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)

{'Cluster': {'ClusterIdentifier': 'sparkify-cluster',
  'NodeType': 'dc2.large',
  'ClusterStatus': 'deleting',
  'ClusterAvailabilityStatus': 'Modifying',
  'MasterUsername': 'quannct',
  'DBName': 'sparkify',
  'Endpoint': {'Address': 'sparkify-cluster.cvidguchcuu7.us-west-2.redshift.amazonaws.com',
   'Port': 5439},
  'ClusterCreateTime': datetime.datetime(2023, 2, 27, 5, 28, 17, 410000, tzinfo=tzutc()),
  'AutomatedSnapshotRetentionPeriod': 1,
  'ManualSnapshotRetentionPeriod': -1,
  'ClusterSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-00dfd05f7ce06ac62',
    'Status': 'active'}],
  'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0',
    'ParameterApplyStatus': 'in-sync'}],
  'ClusterSubnetGroupName': 'default',
  'VpcId': 'vpc-0b20b33ba0fad4013',
  'AvailabilityZone': 'us-west-2a',
  'PreferredMaintenanceWindow': 'mon:06:30-mon:07:00',
  'PendingModifiedValues': {},
  'ClusterVersion': '1.0',
  'AllowVersionUpgrade': True,
  'Number